# Dependencias

In [ ]:
!pip install -q langchain langchain_community tiktoken transformers torch pypdf

In [ ]:
pip install -U bitsandbytes

In [ ]:
!pip install -U datasets

In [ ]:
import os
# optimización experimental de PyTorch que mejora significativamente la gestión
#de memoria GPU y puede reducir el uso de VRAM hasta en 5-6 GiB
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoModel, AutoConfig
import transformers
import torch

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.pdf import PyPDFDirectoryLoader  # Importar cargador de PDFs desde Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Importar separador de texto desde Langchain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [ ]:
from google.colab import files

import gc
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Ecosistema HuggingFace

HuggingFace es tanto una plataforma como un ecosistema centrado en modelos de lenguaje y aprendizaje automático, con énfasis en procesamiento de lenguaje natural (NLP).

Principalmente desarrolla y mantiene la **librería Transformers**, que proporciona acceso a modelos preentrenados (BERT, GPT, T5, etc.).

Este ecosistema facilita la descarga, entrenamiento, fine-tuning y despliegue de modelos de ML existentes dentro de un repositorio para compartir modelos y datasets.

Permite:

- Integración muy rápida de modelos de NLP sin necesidad de entrenar desde cero.
- Experimentación con diversas arquitecturas y tareas diversas (clasificación, generación, traducción, etc.).
- Construcción y despliegue de aplicaciones basadas en LLM.
- Almacenamiento y versionado de modelos y datasets.



**<u>Componentes principales:**</u>

- **Tokenizers:** Implementaciones optimizadas en Rust para tokenización rápida y eficiente.

- **Hub:** Repositorio donde se almacenan modelos y datasets. Permite compartir, descargar y usar modelos preentrenados.

- **Transformers:** Librería con implementaciones de modelos de lenguaje, junto con utilidades para tokenización, entrenamiento y generación.

- **Datasets:** Biblioteca para carga, preprocesamiento y manipulación eficiente de grandes conjuntos de datos.






---
<u>**Registro y obtención del token de acceso en Hugging Face**</u>

Para aprovechar al máximo el ecosistema de Hugging Face (por ejemplo, acceder a modelos privados, usar la API de inferencia o subir modelos/datasets), es recomendable crear una cuenta y generar un token de acceso personal.

###### **Paso a paso para registrarte y obtener tu token:**



1. **Crea una cuenta:**
   - Ve a [https://huggingface.co/join](https://huggingface.co/join) y completa el formulario de registro.
   - Confirma tu correo electrónico siguiendo el enlace que recibirás.

2. **Accede a tu perfil:**
   - Haz clic en tu avatar (arriba a la derecha) y selecciona "Settings" o "Ajustes".

3. **Genera un token:**
   - En el menú lateral, selecciona la opción **Access Tokens**.
   - Haz clic en **New token**.
   - Asigna un nombre descriptivo al token (por ejemplo, `colab-demo`) y elige el permiso `read` (lectura), suficiente para la mayoría de los casos.
   - Haz clic en **Generate**.

4. **Copia el token generado:**
   - Guarda el token en un lugar seguro. **No lo compartas ni lo publiques.**

In [ ]:
# Incluye aqui el token
from google.colab import userdata
HF = userdata.get('HF')
TOKEN = HF
print(TOKEN)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()



El token actúa como credencial segura para interactuar con la plataforma, APIs y recursos asociados.

# Tokenizers

Librería diseñada para realizar tokenización de texto de forma rápida, con alto rendimiento incluso en entornos como notebooks o servidores sin GPU.

Permite convertir texto en tokens el cual incluye la generación de índices numéricos para que puedan ser identificado los tokens y procesados por modelos LLM, y también realizar la operación inversa (decodificación).

- Soporta tokenización preentrenada y entrenamiento desde cero.
- Cada componente puede ser reemplazado o extendido.
- Compatibilidad total con transformers.
- Permite entrenamiento de tokenizadores personalizados desde datasets sin tokenización previa.


<u>Componentes clave:</u>

- Normalizer: Preprocesa texto (lowercase, normalización unicode).
- PreTokenizer: Segmenta texto en unidades básicas.
- Model: Algoritmo de tokenización principal (BPE, WordPiece, Unigram, SentencePiece).
- PostProcessor: Inserta tokens especiales requeridos por el modelo.
- Decoder: Convierte tokens de vuelta a texto.

Compatible totalmente con transformers y permite crear tokenizadores personalizados desde datasets sin tokenización previa.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2',
                                          padding = 'max_length',
                                          truncation = True)
tokenizer

In [ ]:
texts = [
    "Este es un ejemplo de texto para Hamlet.",
    "Hugging Face facilita el uso de modelos preentrenados."
]

#return_tensors='pt', el encoded.input_ids devuelve un tensor 2D donde cada fila representa una secuencia tokenizada
#padding=True, indica que todas las secuencias tokenizadas en un mismo lote deben ser rellenadas (padded) para que tengan la misma longitud que la secuencia más larga del lote
encoded = tokenizer(texts, return_tensors='pt', padding=True)

print(encoded.input_ids)
print(encoded.attention_mask)

---

### Explicación detallada de los tensores en Tokenizers



Al ejecutar el código anterior, el tokenizer devuelve **tensores** (arreglos numéricos).

¿Qué representan estos tensores?

- **`input_ids`:** Son los **índices numéricos** que representan cada token (palabra o subpalabra) en el vocabulario del tokenizer. Cada número corresponde a una entrada específica en el diccionario interno del modelo.

- **`attention_mask`:** Es un tensor binario que indica al modelo qué tokens debe **atender** (valor `1`) y cuáles son padding o tokens irrelevantes (valor `0`).

Una característica de los modelos transformer reside en que **los tensores han de tener la misma dimensión,** esto permite:

- Procesamiento por **lotes (batches)** de secuencias de texto simultáneamente, y para ello, **todas las secuencias deben tener la misma longitud**.
- Si las secuencias tienen longitudes diferentes, se aplica **padding** (relleno) para igualar la longitud al máximo tamaño del lote.
- Esto permite que el tensor sea una **matriz rectangular**, requisito indispensable para el procesamiento eficiente en GPU/CPU.

### ¿Para qué sirve decodificar?





Decodificar tokens permite devolver la expresión númerica a texto, es útil por tanto para:

- Debugging: Verificar si el tokenizer está dividiendo las palabras correctamente
- Comprensión: Entender cómo el modelo "ve" tu texto
- Optimización: Detectar si hay problemas de tokenización que afecten el rendimiento
- Aprendizaje: Visualizar el proceso de tokenización para comprender mejor los transformers

In [ ]:
all_tokens = [tokenizer.convert_ids_to_tokens(seq) for seq in encoded.input_ids]

print("Tokens de todas las secuencias:")
for i, tokens in enumerate(all_tokens):
  print(f"Secuencia {i+1}: {tokens}")


### ¿Qué significan los tokens que ves tras la tokenización?




Cuando tokenizas un texto usando modelos como BERT o derivados, aparecen tokens especiales y subpalabras.

<u>**Tokens especiales**</u>

- **[CLS]**    **Classification**  
  Marca el **inicio** de cada secuencia. Es utilizado por el modelo para tareas de clasificación de frases completas.

- **[SEP]**  **Separator**  
  Marca el **final** de cada secuencia. También se usa para separar frases en tareas como preguntas y respuestas.

- **[PAD]**  **Padding**  
  Es un **relleno** que se agrega para que todas las secuencias tengan la misma longitud dentro de un batch. Los modelos ignoran estos tokens durante el cálculo.

<u>**Tokens con `##`**</u>

Los tokens que comienzan con `##` indican que son **continuaciones de palabras** (subword tokenization).  
Esto ocurre porque el modelo utiliza un vocabulario limitado de subpalabras para poder manejar palabras desconocidas o poco frecuentes.

Permite que el modelo entienda y procese palabras nuevas dividiéndolas en partes conocidas.
**¿Por qué se hace esto?**

- Los **tokens especiales** ayudan al modelo a entender la estructura de la entrada.
- La **tokenización en subpalabras** permite manejar vocabularios más pequeños y cubrir palabras desconocidas de forma eficiente.

---

# Modelos


**Model Hub:** Busca democratizar el acceso a modelos de ML y acelerar su adopción.


- Almacena y versiona modelos entrenados por la comunidad y por Hugging Face.
- Permite descarga directa e integración inmediata con librerías como transformers.
- Soporta modelos en distintos formatos y frameworks (PyTorch, TensorFlow, etc.).
- Incluye metadatos, documentación, y archivos auxiliares (configuraciones, tokenizadores).

Puedes consultar los modelos disponibles dentro del Hub aquí: https://huggingface.co/models

`AutoModel` se refiere a la clase base en Hugging Face Transformers que carga un modelo preentrenado sin tareas específicas, útil para embeddings o fine-tuning general. En cambio, `AutoModelForCausalLM`, `AutoModelForSequenceClassification`, etc., son subclases especializadas (`AutoModel**`) que cargan modelos ajustados para tareas concretas como lenguaje causal, clasificación o preguntas y respuestas.


Permite cargar modelos directamente en código:

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

Si no tenemos una GPU o no tenemos espacio como para alojar el modelo, podemos **cuantizarlos.** Cuantizar un modelo significa restarle precisión a cada uno de sus parámetros para que pese menos. **Habrá que llegar a un trade-off de modelo pequeño con sus parámetros intactos vs modelo grande con menor precisión.**


__*OJO: No todos los modelos se pueden cuantizar*__

Para poder cuantizar un modelo:

In [ ]:
model_id = 'mistralai/Mistral-7B-Instruct-v0.2'
device = "cuda"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id,device_map = device, torch_dtype=torch.bfloat16)

In [ ]:
# Codigo que nos permite limpiar la memoria RAM
gc.collect()
torch.cuda.empty_cache()

Establecemos la configuracón para que cargue el modelo consumiendo menos memoria.

In [ ]:
#load_in_4bit=True
#Función: Habilita la cuantización a 4 bits del modelo
#Beneficio: Reduce drásticamente el uso de memoria (aproximadamente 75% menos que float32)
#Alternativa: load_in_8bit=True para cuantización a 8 bits (menos ahorro pero mayor precisión)

#bnb_4bit_quant_type='nf4'
#Función: Especifica el tipo de cuantización a usar
#NF4 (Normal Float 4): Tipo de dato optimizado para pesos que siguen una distribución normal
#Alternativa: 'fp4' (Float Point 4) - menos eficiente que NF4

#bnb_4bit_use_double_quant=True
#Función: Aplica cuantización anidada (nested quantization)
#Beneficio: Ahorra 0.4 bits adicionales por parámetro sin pérdida de rendimiento
#Proceso: Realiza una segunda cuantización sobre los pesos ya cuantizados
#Resultado: Permite entrenar modelos como Llama-13b en una GPU de 16GB

#bnb_4bit_compute_dtype=torch.bfloat16
#Función: Define el tipo de dato para los cálculos internos
#Importante: Aunque los pesos se almacenan en 4-bit, los cálculos se hacen en 16-bit
#Beneficio: bfloat16 es más rápido que float32 (valor por defecto)
#Compatibilidad: Mejor para GPUs modernas que soportan bfloat16

'''bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)'''

# Configuración más agresiva para ahorrar memoria
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,  # Cambiar a float16 en lugar de bfloat16
    bnb_4bit_device_map="auto"
)


In [ ]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
model_config = AutoConfig.from_pretrained(
    model_id
)

#trust_remote_code=True
#Función: Permite ejecutar código personalizado del modelo remoto
#Uso: Necesario para algunos modelos que tienen implementaciones personalizadas

#config=model_config
#Función: Usa la configuración específica del modelo
#Beneficio: Asegura compatibilidad con la arquitectura original

#device_map=device
#Función: Especifica en qué dispositivo cargar el modelo
#Opciones: "cuda", "cpu", "auto" (distribución automática)

#torch_dtype=torch.bfloat16
#Función: Define el tipo de dato para los módulos no cuantizados
#Coherencia: Debe coincidir con bnb_4bit_compute_dtype para consistency

model_cuant = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map=device,
    torch_dtype=torch.bfloat16,
    quantization_config = bnb_config
    )

In [ ]:
def get_model_precision(model):
    precision = set()
    for param in model.parameters():
        precision.add(param.dtype)
    return precision

def get_model_size(model):
    param_size = 0
    buffer_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_gb = (param_size + buffer_size) / 1024**3
    return size_all_gb

In [ ]:
print(f"Size of model parameters: {get_model_size(model):.2f} GB")

In [ ]:
#Lo dividimos entre 2 porque lo estamos cargando en 4 bits
print(f"Size of model parameters: {get_model_size(model_cuant):.2f} GB")

In [ ]:
print("Model precision:", get_model_precision(model_cuant))

Vemos que lo que pesa el modelo no es exactamente el peso del anterior dividido entre 4 debido a que hay parámetros que no se han podido cuantizar. Pero si lo hemos reducido considerablemente

# Transformers

Librería de código abierto que facilita el uso de modelos de **Deep Learning** basados en la arquitectura **Transformer**. Proporciona acceso a una  colección de modelos preentrenados para tareas de procesamiento de lenguaje natural (PLN), tales como: clasificación de texto, generación de texto, traducción automática, resumen, respuesta a preguntas, etcétera.

Entre sus funcionalidades destacan:

- Carga automática de modelos y tokenizadores desde el repositorio HuggingFace Hub.
- Soporte para múltiples frameworks de backend: PyTorch, TensorFlow y JAX.
- Compatibilidad con entrenamiento, fine-tuning e inferencia en CPU, GPU y entornos distribuidos.
- Interfaz pipeline para ejecutar tareas complejas con pocas líneas de código.
- Extensibilidad para integrar modelos personalizados, adaptar tareas o modificar arquitecturas.

En resumen, permite conectar modelos a flujos de preprocesamiento y posprocesamiento, aceptando texto directamente como entrada y devolviendo resultados estructurados.

Puedes consultar más información aquí:
- https://huggingface.co/docs/hub/transformers
- https://github.com/huggingface/transformers




### Tipos de arquitecturas en Transformers




Los modelos basados en la arquitectura Transformer pueden clasificarse en tres grandes grupos, según las partes que utilizan. Elegir el modelo adecuado para cada tarea mejora la eficiencia y la calidad de los resultados:

**1. Encoders (solo codificador)**
- Procesan la **entrada completa** de manera bidireccional y generan una representación contextualizada de cada token.
- **Ejemplos:** BERT, RoBERTa, DistilBERT.
- **Tareas típicas:** clasificación de texto, análisis de sentimiento, extracción de entidades, embeddings.

**2. Decoders (solo decodificador)**
- Generan texto de manera **autoregresiva** (palabra por palabra), usando solo el contexto previo.
- **Ejemplos:** GPT, GPT-2, GPT-3.
- **Tareas típicas:** generación de texto.

**3. Encoder-Decoder (sequence-to-sequence, seq2seq)**
- Combinan un encoder (que procesa la entrada) y un decoder (que genera la salida).
- Permiten transformar una secuencia de entrada en una secuencia de salida de diferente longitud.
- **Ejemplos:** T5, BART, MarianMT.
- **Tareas típicas:** traducción automática, resumen de textos, generación de qa, data-to-text.

| Arquitectura       | Ejemplo de modelo | Tareas principales                    |
|--------------------|------------------|---------------------------------------|
| Encoder            | BERT, RoBERTa    | Clasificación, embeddings, NER        |
| Decoder            | GPT-2, GPT-3     | Generación de texto                   |
| Encoder-Decoder    | T5, BART         | Traducción, resumen, QA generativa    |


## pipeline()

Dentro de la librería **transformers**, el objeto más básico es **pipeline()**. Permite utilizar modelos preentrenados creando una interfaz de alto nivel para ejecutar tareas de procesamiento de lenguaje natural sin necesidad de configurar manualmente el modelo, el tokenizador ni la lógica de inferencia.

Es decir, abstrae el proceso de carga del modelo preentrenado correspondiente a la tarea especificada (por ejemplo, "text-generation", "summarization", "translation", etc.), junto con su tokenizador asociado, y devuelve un objeto que acepta texto como entrada y devuelve la salida del modelo en formato estructurado.


`pipeline` **carga automáticamente un modelo predefinido por defecto** para cada  tarea si no especificas uno explícito, pero se le puede indicar la carga de un modelo concreto.

In [ ]:
help(pipeline)

El modelo se descarga y se almacena en el caché cuando creas el objeto.

Existen multitud de tareas que pueden lanzarse:

<u>**Tareas de texto**</u>
- **text-classification (sentiment-analysis):** clasificación de texto en categorías predefinidas (sentimiento, tema, intención, etc.).
- **text-generation:** generación de texto a partir de un prompt. Se usa para diálogo, escritura asistida, etc.
- **text2text-generation:** generación de texto mediante modelos encoder-decoder. Input y output son texto (T5, BART).
- **fill-mask:** predicción de tokens enmascarados ([MASK]) en una secuencia. Útil para completar frases.
- **summarization:** condensación de textos largos manteniendo la información esencial.
- **translation / translation_xx_to_yy:** traducción automática entre idiomas.
- **question-answering:** extracción de respuestas a partir de un contexto y una pregunta (SQuAD-style).
- **table-question-answering:** responde preguntas basadas en tablas estructuradas (modelos como TAPAS).
- **token-classification (ner):** etiquetado de entidades nombradas (personas, lugares, organizaciones, etc.).
- **feature-extraction:** obtención de representaciones vectoriales de texto (embedding para downstream tasks).

Pero no solamente se centran en tareas de texto, existen <u>tareas de audio</u> (**audio-classification**, **text-to-audio**, etc.), <u>tareas de imagen</u> (**image-classification**, **image-to-text**, etc,), <u>tareas de video</u> (**video-classification**) y <u>tareas multimodales</u> (**image-text-to-text**,...)


### Casos de uso

#### **1.Análisis de sentimiento**

Implementa una forma especializada de "text-classification" para detectar la carga emocional o actitudinal de un texto, generalmente clasificándolo en categorías como: **positivo**, **negativo** y a veces **neutral**.

Utilizando un Encoder

<u>Funcionamiento:</u>

- Tokenización del texto de entrada.
- Inferencia con modelo de clasificación de secuencias (normalmente BERT, RoBERTa, DistilBERT, etc.) que ha sido afinado con datasets etiquetados con emociones o polaridad (como SST-2 o IMDb).
- Decodificación de logits en etiquetas como POSITIVE, NEGATIVE, con un valor de confianza (score).

<u>Aplicaciones:</u>

- Análisis de feedback de clientes.
- Monitorización de marca en redes sociales.
- Clasificación de reseñas.
- Evaluación de respuestas automatizadas.

In [ ]:
classifier = pipeline("sentiment-analysis")


In [ ]:
classifier("No se ha podido entregar el paquete")

In [ ]:
classifier('Hola, ¿Cómo estás?')

In [ ]:
classifier(
    ["Hola, estoy escribiéndoles porque he detectado un cargo no autorizado en mi tarjeta de crédito por un importe elevado. Esto ocurrió el día 10 de julio y no reconozco esa transacción en absoluto. No he compartido mis datos con nadie, y mi tarjeta no fue extraviada ni utilizada por terceros, así que no entiendo cómo se generó dicho cargo. Solicito que revisen esto con urgencia y que me devuelvan el dinero cuanto antes. Ya he tenido otros problemas con la cuenta, y necesito una solución definitiva. Agradezco su pronta respuesta.",
     "Llevo semanas intentando resolver un problema con un cargo no reconocido en mi tarjeta de crédito. Ya envié toda la documentación requerida."]
)

Como comentábamos al principio, `pipeline()` carga por defecto un modelo por caso de uso, pero pero también puedes escoger un modelo particular del Hub y usarlo en un pipeline para una tarea específica. Es esperable que el potencial sentimiento detectado varíe entre distintos modelos.


In [ ]:
clas = pipeline("sentiment-analysis",model="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
clas('Hola, ¿Cómo estás?')

In [ ]:
clas(
    ["Hola, estoy escribiéndoles porque he detectado un cargo no autorizado en mi tarjeta de crédito por un importe elevado. Esto ocurrió el día 10 de julio y no reconozco esa transacción en absoluto. No he compartido mis datos con nadie, y mi tarjeta no fue extraviada ni utilizada por terceros, así que no entiendo cómo se generó dicho cargo. Solicito que revisen esto con urgencia y que me devuelvan el dinero cuanto antes. Ya he tenido otros problemas con la cuenta, y necesito una solución definitiva. Agradezco su pronta respuesta.",
     "Llevo semanas intentando resolver un problema con un cargo no reconocido en mi tarjeta de crédito. Ya envié toda la documentación requerida."]
)

En ciertos contextos puede ser una buena idea combinar los resultados de varios modelos, puesto que:

- Incrementa la robustez ante dominios distintos:
Combinar modelos entrenados en diferentes dominios puede mejorar la cobertura.
Un modelo afinado en reseñas de cine puede no generalizar bien a textos financieros.

- Reduce sesgo:
Distintos modelos pueden tener sesgos distintos (léxicos, culturales, etc.). Un enfoque de agregación (por ejemplo, votación o promediado de scores) puede disminuir estos efectos.

- Multi-clase vs binario:
Algunos modelos detectan positivo/negativo; otros añaden neutral, sarcasmo o intensidad. Combinarlos permite enriquecer la clasificación.

- Meta-clasificadores (ensemble):
Los outputs de varios modelos pueden alimentarse a un clasificador adicional que aprenda a decidir mejor en base a las salidas combinadas.

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Prueba a usar un modelo de clasificación de texto diferente y comprueba las diferencias en las clasificaciones y los scores.



#### **2.Text generation**

Permite simplificar la generación automática de texto usando modelos autoregresivos (como GPT-2, GPT-3, LLaMA, etc.).

Utilizando Decoders

In [ ]:
#text_model = "mistralai/Mistral-Small-24B-Instruct-2501"
text_model = "openai-community/gpt2-large"

# Cargar pipeline de generación de texto con un modelo ligero para ejemplo
generator = pipeline("text-generation",
                     model=text_model,
                     max_length=100,
                     repetition_penalty = 1,
                     temperature =0.1)


In [ ]:
# Prompt específico para contexto bancario
prompt = (
    "El cliente solicitó un préstamo personal para la compra de un vehículo. "
    "El monto aprobado fue de 15,000€ con un plazo de "
)


In [ ]:
# Generar texto completando el prompt
result = generator(prompt, num_return_sequences=100)

# Mostrar texto generado
print(result[0]['generated_text'])


In [ ]:
gc.collect()
torch.cuda.empty_cache()

#### **3.Summarization**

Esta funcionalidad permite realizar una síntesis rápida de documentos extensos, así como generación de extractos para informes o reúmenes automáticos.
Para ello:
- Recibe un texto de entrada largo.
- El encoder procesa el texto para obtener una representación contextual.
- El decoder genera un texto más corto que sintetiza la información relevante.
- El pipeline devuelve este resumen en texto natural.

Utilizando Encoder-Decoder o modelos sequence-to-sequence

In [ ]:
#summarizer = pipeline("summarization", model = "facebook/bart-large-cnn")
summarizer = pipeline("summarization",
                      model = "google/pegasus-cnn_dailymail",
                      max_length=128,
                      min_length=15,
                      do_sample=False,
                      temperature=0.5,
                      top_k=50,
                      top_p=0.9)




In [ ]:
documento = """
La banca digital sigue transformando la forma en que los clientes interactúan con sus entidades financieras.
En el último trimestre, se ha observado un incremento del 35% en el uso de la app móvil para transferencias
y pagos. Además, la incorporación de sistemas biométricos ha permitido reducir los intentos de fraude en
un 20%. Por otro lado, el nuevo sistema de asesoramiento automatizado basado en inteligencia artificial
ha comenzado a ofrecer recomendaciones personalizadas sobre inversión y ahorro a más de 15.000 usuarios.
Estas mejoras apuntan a una estrategia de digitalización centrada en la experiencia del cliente y en la seguridad de las operaciones.
"""

In [ ]:
resumen = summarizer(documento)[0]["summary_text"]
print("Resumen:", resumen)

In [ ]:
# Codigo que nos permite limpiar la memoria RAM
gc.collect()
torch.cuda.empty_cache()

Apliquemos un modelo Encoder-Decoder

In [ ]:
model = "google/pegasus-cnn_dailymail"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

In [ ]:

inputs = tokenizer(documento, truncation=True, padding="longest", return_tensors="pt").to(device)
summary_ids = model_pegasus.generate(**inputs,
                                     max_length=128,
                                     min_length=30,
                                     length_penalty=2.0,
                                     num_beams=4,
                                     early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

#### 4.Feature extraction y vector positioning


Extrae representaciones vectoriales densas (embeddings) que codifican la semántica de textos, imágenes u otros datos en un espacio continuo. Luego, al posicionar una consulta en ese mismo espacio, se mide la proximidad o similitud con otros vectores para recuperar los elementos más relevantes en función del significado compartido.

Utilizando Encoders

In [ ]:
# Modelo para extracción de features (embeddings)
model_name = "sentence-transformers/all-MiniLM-L6-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


In [ ]:

def embed_text(text):
    # Tokenizar con truncamiento y padding
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Obtener embeddings promedio de la última capa oculta (mean pooling)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

In [ ]:
# Corpus de ejemplo (documentos bancarios)
documents = [
    "Cuenta de ahorro con tasa anual del 2%.",
    "Servicio de banca móvil disponible 24/7.",
    "Préstamos personales con tasa fija.",
    "Atención al cliente de lunes a viernes."
]

# Obtener vectores de documentos
doc_vectors = np.vstack([embed_text(doc) for doc in documents])

# Consulta de ejemplo
query = "¿Qué tasa ofrece la cuenta de ahorro?"
query_vec = embed_text(query)

# Calcular similitud coseno entre consulta y documentos
similarities = cosine_similarity(query_vec, doc_vectors).flatten()

In [ ]:
# Ordenar documentos por similitud descendente
ranking = similarities.argsort()[::-1]

print("Consulta:", query)
print("\nDocumentos ordenados por similitud:")

for idx in ranking:
    print(f"- {documents[idx]} (similitud: {similarities[idx]:.4f})")


#### **5.Token-classification (extracción de entidades)**

 Ejecuta token a token la clasificación de etiquetas predefinidas, agrupando palabras contiguas que forman entidades. Tambien se le conoce por el nombre de **Named Entity Recognition (NER)**.

 Utilizando Encoders

In [ ]:

ner_model = "dslim/bert-base-NER"

ner = pipeline("token-classification", model=ner_model, aggregation_strategy="simple")

# Texto a analizar
text = "El cliente Juan Pérez solicitó un préstamo hipotecario de 250,000 euros el 15 de marzo de 2023 en la sucursal de la calle Santa Engracia 179, en Madrid."

# Ejecutar el pipeline
entities = ner(text)




In [ ]:
entities

In [ ]:
# Mostrar entidades detectadas
for entity in entities:
    print(f"{entity['word']}: {entity['entity_group']} ({entity['score']:.2f})")

Las etiquetas predefinidas dependen del modelo usado. En este caso, este modelo está entrenado para reconocer 4 tipos de entidades:
- Localización (LOC).
- Organizaciones (ORG).
- Persona (PER).
- Misceláneo (MISC) como categoría genérica para el resto de entidades.

Puedes obtener información sobre qué etiquetas existen directamente del modelo:

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(ner_model)
print(config.id2label)

Es posible agregar etiquetas propias, pero para ello es necesario aplicar fine-tuning de un modelo preentrenado con un dataset anotado con las etiquetas específicas.

Para ello necesitas:
- Conjunto de datos con texto y etiquetas tokenizadas según tu esquema personalizado (por ejemplo, CLIENTE, PRODUCTO, IMPORTE, etc.).

- Entrenamiento un modelo preexistente para que aprenda a clasificar tokens según esas etiquetas.

El pipeline token-classification acepta estos modelos personalizados sin cambios en la interfaz.

In [ ]:
gc.collect()
torch.cuda.empty_cache()

####  **6.Zero-shot classification**

Ejecuta clasificación de texto **sin entrenamiento previo** en las categorías específicas que deseas usar. Simplemente proporcionas el texto y las posibles etiquetas, y el modelo predice la categoría más apropiada. También se le conoce por el nombre de **Zero-shot Text Classification**.

Esta técnica es especialmente útil cuando:
- No tienes datos etiquetados para entrenar un clasificador
- Quieres probar rápidamente diferentes categorías
- Necesitas flexibilidad para cambiar las etiquetas sin reentrenar



Sequence to sequence

In [ ]:
# Cargar el pipeline de zero-shot classification
zeroshot_model = "facebook/bart-large-mnli"
classifier = pipeline("zero-shot-classification", model=zeroshot_model)

In [ ]:
# Texto a clasificar
text = "El cliente XYZ ha solicitado el reembolso de un cargo no autorizado en su tarjeta de crédito"
candidate_labels = ["fraude", "consulta general", "reclamación", "cierre de cuenta"]
result = classifier(text, candidate_labels)

In [ ]:
# Mostrar resultados de manera más clara
print("Resultados de clasificación zero-shot:")
print(f"Texto: '{text}'\n")

for label, score in zip(result['labels'], result['scores']):
    print(f"{label.upper()}: {score:.4f} ({score*100:.2f}%)")



El modelo devuelve las etiquetas **ordenadas por probabilidad**, donde:
- **Puntuación más alta** = categoría más probable
- **Suma de todas las puntuaciones** = 1.0 (100%)
- El modelo **no fue entrenado específicamente** en estas categorías, pero puede inferir el contenido


**Clasificación multi-etiqueta**

Puedes permitir que el texto pertenezca a **múltiples categorías** simultáneamente:


In [ ]:
complex_text = """
El cliente reportó problemas al acceder a la app móvil tras la última actualización del sistema.
Además, menciona que no puede visualizar el historial de movimientos ni confirmar transferencias
programadas. Ha solicitado asistencia técnica urgente.
"""


business_labels = [
    "problemas técnicos",
    "seguridad",
    "servicio al cliente",
    "banca móvil",
    "operaciones fallidas",
    "incidencias"
]

# Clasificación multi-etiqueta
multi_result = classifier(
    complex_text,
    business_labels,
    multi_label=True  # Permite múltiples etiquetas
)

print("Clasificación multi-etiqueta:")
print(f"Texto: {complex_text[:100]}...\n")

for label, score in zip(multi_result['labels'], multi_result['scores']):
    if score > 0.5:  # Solo mostrar etiquetas con alta confianza
        print(f"{label.upper()}: {score:.4f} ({score*100:.2f}%)")
    else:
        print(f"{label}: {score:.4f} ({score*100:.2f}%)")


In [ ]:
# Limpiar memoria
gc.collect()
torch.cuda.empty_cache()

#### **7.Question Answering**

Permite construir sistemas de respuesta automática a preguntas basadas en un contexto textual. Utiliza **modelos extractivos** que identifican el fragmento de texto más probable como respuesta a una pregunta, dentro de un contexto dado.

Estos modelos están basados en arquitecturas decoders como BERT, RoBERTa o DistilBERT, etcétera. **No generan texto nuevo, sino que extraen la respuesta del contexto suministrado.**

Funcionamiento:

- Una pregunta (question).

- Un contexto (context) que contiene la información relevante.

- El modelo codifica ambos mediante atención cruzada y asigna una probabilidad a cada token como posible inicio y fin de la respuesta. Finalmente devuelve el span más probable del contexto como respuesta.

Es útil para textos cortos y cuando el contexto está controlado y contiene la respuesta exacta.

La principal diferencia frente a la utilización de RAG, reside en que no existe recuperación documental y generación neural en pasos separados.

In [ ]:
qa_model = pipeline("question-answering",
                    handle_impossible_answer=1,
                    max_answer_len=500,
                    max_seq_len=300)

In [ ]:


context = """
La entidad bancaria BBVA ofrece cuentas de ahorro y cuentas corrientes. Para abrir una cuenta, el cliente debe presentar una identificación oficial y comprobante de domicilio.
Las tasas de interés para cuentas de ahorro varían entre 0.01% y 2.7% anual. Los clientes pueden acceder a servicios digitales como banca móvil y transferencias electrónicas.
El horario de atención es de lunes a viernes de 9:00 a 15:00 horas en oficina bancaria y las 24h en canales online.
"""

question_1 = "¿Qué documentos necesito para abrir una cuenta en el banco BBVA?"
question_2 = "¿Cuál es el horario de atención al cliente?"




In [ ]:
respuesta_1 = qa_model(question=question_1, context=context)
respuesta_2 = qa_model(question=question_2, context=context)

print("Pregunta:", question_1)
print("Respuesta:", respuesta_1['answer'])

print("\nPregunta:", question_2)
print("Respuesta:", respuesta_2['answer'])

In [ ]:
gc.collect()
torch.cuda.empty_cache()

# Datasets

La librería `datasets` proporciona acceso estandarizado a miles de datos para usar en tareas de NLP, visión, audio y multímodales. Esta librería está optimizada para streaming, eficiencia de memoria y escalabilidad.

Colección de Datasets disponibles en: https://huggingface.co/datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("ag_news", download_mode="force_redownload")


In [ ]:
dataset["train"]

In [ ]:
df = dataset["train"].to_pandas()
df.head()

A su vez, permite trabajar con datasets grandes sin descargarlos completamente:

In [ ]:
# Cargar dataset financiero con apodo de sentimiento en modo streaming
dataset = load_dataset("mltrev23/financial-sentiment-analysis", streaming=True)

In [ ]:
for i, example in enumerate(dataset["train"]):
    print(f"TÍTULO: {example['Sentence']}")
    print(f"SENTIMIENTO: {example['Sentiment']}")
    if i == 4:
        break

# Langchain + HuggingFace

Tal y como hemos visto en el notebook anterior, LangChain es un framework para construir aplicaciones con modelos de lenguaje, proporcionando abstracciones para manejar cadenas de llamadas, memoria, agentes y fuentes externas.

Por su parte, HuggingFace ofrece modelos y pipelines de NLP preentrenados, accesibles desde su librería transformers.

La combinación permite **usar modelos HuggingFace dentro de LangChain** para tareas como generación, resumen, pregunta-respuesta, etcétera, con integración sencilla en flujos de trabajo complejos.

LangChain provee wrappers específicos (HuggingFaceHub, HuggingFacePipeline) para conectar modelos HuggingFace como componentes dentro de cadenas o agentes.

Esto facilita:

- Invocar modelos HuggingFace en arquitecturas modulares.
- Combinar múltiples modelos y fuentes de datos.
- Añadir memoria, prompt templates, y lógica de control.

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model_summary = "facebook/bart-large-cnn"

# Crear pipeline HuggingFace para resumen
summarizer = pipeline("summarization", model=model_summary)

# Adaptar pipeline para LangChain
llm = HuggingFacePipeline(pipeline=summarizer,
                          model_kwargs={"temperature": 0.7})

# Definir plantilla de prompt para LangChain
prompt_template = """
Resume el siguiente texto de forma clara y concisa:

{document}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["document"])

# Crear cadena LangChain con LLM y prompt
chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:


# Texto a resumir (ejemplo)
document = """
La banca digital ha revolucionado la forma en que los usuarios interactúan con los servicios financieros.
La adopción de tecnologías digitales permite una experiencia más rápida, segura y personalizada.
Las plataformas digitales ofrecen acceso 24/7, eliminando barreras geográficas y temporales.
Los clientes pueden realizar transferencias, pagos y consultas sin necesidad de acudir a sucursales físicas.
Además, la inteligencia artificial y el análisis de datos mejoran la detección de fraudes y la personalización de productos financieros.
La banca digital también fomenta la inclusión financiera, llegando a poblaciones previamente desatendidas.
Sin embargo, plantea desafíos en ciberseguridad, privacidad y adaptación regulatoria, que las entidades deben gestionar rigurosamente para mantener la confianza del usuario.
"""

# Ejecutar cadena
resumen = chain.run(document)
print("Resumen generado:")
print(resumen)


Veamos ahora otro ejemplo donde aplicaremos argumentos específicos tanto para HuggingFace como para LangChain.

In [ ]:

# Pipeline HuggingFace para generación de texto con parámetros de generación
text_gen_pipeline = pipeline(
    "text-generation",
    model=model_cuant,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.95,
    do_sample=True,
    pad_token_id=50256,
    eos_token_id=50256
)

# Adaptar pipeline a LangChain con argumentos específicos
llm = HuggingFacePipeline(
    pipeline=text_gen_pipeline,
    model_kwargs={}
)




In [ ]:
# Prompt template con variable
prompt_template = PromptTemplate(
    template = """Explica brevemente a un cliente bancario {topic}. Usa un lenguaje sencillo y claro.""",
    input_variables=["topic"]
)

# Crear cadena LangChain
chain = LLMChain(llm=llm, prompt=prompt_template)

# Ejecutar cadena
respuesta = chain.run({"topic": "La importancia de ahorrar."})
print(respuesta)